# <center> <font style="color:rgb(100,109,254)"> Optical Flow </font> </center>
Optical flow is a motion estimation algorithm. This will help you figure out apparent motion of image objects between two consecutive frames caused by either movement of object or camera. 

Optical flow has many applications in areas like:

- Object Tracking

- Structure from Motion (SFM)

- Video Compression

- Visual Odometry 

- Video Stabilization.


**Following two constraints must be taken into account while using Optical Flow:**

1. The pixel intensities of an object do not change between consecutive frames.
2. Neighbouring pixels have similar motion.



In [5]:
import cv2
import numpy as np

## <font style="color:rgb(134,19,348)">  Tracking Points with Lucas-Kanade Optical Flow.  </font>
Now we are going to try Lucas-Kanade version of Optical flow, which is a sparse Optical flow implementation. Since we are just going to track some points in a video, we should use good feature points and some good points to track with Optical flow are corners, so we're going to use `cv2.goodFeaturesToTrack()` to get corners and then track them using the function `cv2.calcOpticalFlowPyrLK()`.

[```nextPts, status, err = cv2.calcOpticalFlowPyrLK(prevImg, nextImg, prevPts, nextPts[, status[, err[, winSize[, maxLevel[, criteria]]]]])```](https://docs.opencv.org/4.2.0/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323)

Params:

- **`prevImg`**	first 8-bit input image or pyramid constructed by buildOpticalFlowPyramid.
- **`nextImg`**	second input image or pyramid of the same size and the same type as prevImg.
- **`prevPts`**	vector of 2D points for which the flow needs to be found; point coordinates must be single-precision floating-point numbers.
- **`nextPts`**	output vector of 2D points (with single-precision floating-point coordinates) containing the calculated new positions of input features in the second image; when OPTFLOW_USE_INITIAL_FLOW flag is passed, the vector must have the same size as in the input.
- **`status`**	output status vector (of unsigned chars); each element of the vector is set to 1 if the flow for the corresponding features has been found, otherwise, it is set to 0.
err	output vector of errors; each element of the vector is set to an error for the corresponding feature, type of the error measure can be set in flags parameter; if the flow wasn't found then the error is not defined (use the status parameter to find such cases).
- **`winSize`**	size of the search window at each pyramid level.
- **`maxLevel`**	0-based maximal pyramid level number; if set to 0, pyramids are not used (single level), if set to 1, two levels are used, and so on; if pyramids are passed to input then algorithm will use as many levels as pyramids have but no more than maxLevel.
- **`criteria`**	parameter, specifying the termination criteria of the iterative search algorithm (after the specified maximum number of iterations criteria.maxCount or when the search window moves by less than criteria.epsilon.


In [8]:
# Reading a Sample Video
cap = cv2.VideoCapture('media/M4/slow.mp4')

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),  maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Take first frame as the initial frame and find corners (good point to track) in it
ret, frame = cap.read()

# Convert to grayscale
previous_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Get the corner points
old_points = cv2.goodFeaturesToTrack(previous_frame, mask = None, maxCorners = 200, qualityLevel = 0.2, minDistance = 5,
blockSize = 7)

# Create a black image, this will be our drawing Canvas.
mask = np.zeros_like(frame)

while(True):
    
    ret, frame = cap.read()
    if not ret:
        break
        
    current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate optical flow
    new_points, status, err = cv2.calcOpticalFlowPyrLK(previous_frame, current_frame, old_points, None, **lk_params)
   
    # Select good points based on the status
    new_good_points = new_points[status==1]
    old_good_points = old_points[status==1]

    for (new, old) in zip(new_good_points, old_good_points):
        a,b =  new.ravel()
        c,d = old.ravel()
        # Draw a line from x1,y1 to x2,y2
        mask = cv2.line(mask, (a,b),(c,d), [0,0,255], 2)
        # Circle the tracked Corner Points
        frame = cv2.circle(frame,(a,b),5,[255,0,0],-1)     
        
    # Combine the mask and frame to display the drawing.    
    combined = cv2.add(frame,mask)
    cv2.imshow('frame', combined)

    k = cv2.waitKey(30) 
    if k == 27:
        break
        
    # At the end current frame becomes the previous frame for the next iteration
    previous_frame = current_frame.copy()
    
    # At the end new points become the old points for the next iteration
    old_points = new_good_points.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

## <font style="color:rgb(134,19,348)">   Tracking a Single Point with Optical Flow </font>
With below script you can choose a single point to track with the mouse on a webcam feed.

In [9]:
cap = cv2.VideoCapture(1)
 
_, frame = cap.read()
frame = cv2.flip(frame, 1) 

prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
# Lucas kanade params
lk_params = dict(winSize = (10, 10), maxLevel = 4,  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
 
# This function will let user select a point to track.
def select_point(event, x, y, flags, params):
    global point, point_selected, old_point
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        point_selected = True
        old_point = np.array([[x, y]], dtype=np.float32).reshape(-1,1,2)
 
cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
cv2.setMouseCallback("Frame", select_point)

# By default this would be False.
point_selected = False

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    frame = cv2.flip( frame, 1 ) 
    current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
    # If a point was selected by the mouse then proceed.
    if point_selected:
        #cv2.circle(frame, point, 5, (0, 0, 255), 2)

        new_point, status, error = cv2.calcOpticalFlowPyrLK(prev_frame, current_frame, old_point, None, **lk_params)
        
        if status[0][0] > 0:
        
            new_good_point = new_point[status==1]

            x, y = new_good_point.ravel()
            cv2.circle(frame, (x, y), 10, (0, 0, 255), -1)

            prev_frame = current_frame
            old_point = new_good_point.reshape(-1,1,2)
        else:
            cv2.putText(frame,'Lost Point',(10,50), cv2.FONT_HERSHEY_COMPLEX, 1, (20,25,150), 2, cv2.LINE_AA)
 
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
 
cap.release()
cv2.destroyAllWindows()

## <font style="color:rgb(134,19,348)"> Dense Optical Flow in OpenCV   </font>
Lucas-Kanade method computes optical flow for a sparse feature set (which we used as corners). OpenCV provides another algorithm to find dense optical flow. It computes the optical flow for all the points in the frame. It is based on **Gunner Farneback's** algorithm which is explained in [**`Two-Frame Motion Estimation Based on Polynomial Expansion" by Gunner Farneback in 2003.`**](http://www.diva-portal.org/smash/get/diva2:273847/FULLTEXT01.pdf)

**You can read more about this by reading the research paper linked above**

Below python script shows how to find the dense optical flow using Gunner's algorithm. We get a 2-channel array with optical flow vectors, (u,v) using the function **cv2.calcOpticalFlowFarneback()**. We then find their magnitude and direction. We color code the result for better visualization. Direction corresponds to Hue value of the image. Magnitude corresponds to Value plane. Saturation is kept maximum (255) for maximizing visibility.

[```flow	= cv2.calcOpticalFlowFarneback(	prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags)```](https://docs.opencv.org/4.2.0/dc/d6b/group__video__track.html#ga5d10ebbd59fe09c5f650289ec0ece5af)

**Params:**

- **`prev`**	first 8-bit single-channel input image.
- **`next`**	second input image of the same size and the same type as prev.
- **`flow`**	computed flow image that has the same size as prev and type CV_32FC2.
- **`pyr_scale`**	parameter, specifying the image scale (<1) to build pyramids for each image; pyr_scale=0.5 means a classical pyramid, where each next layer is twice smaller than the previous one.
- **`levels`**	number of pyramid layers including the initial image; levels=1 means that no extra layers are created and only the original images are used.
- **`winsize`**	averaging window size; larger values increase the algorithm robustness to image noise and give more chances for fast motion detection, but yield more blurred motion field.
iterations	number of iterations the algorithm does at each pyramid level.
- **`poly_n`**	size of the pixel neighborhood used to find polynomial expansion in each pixel; larger values mean that the image will be approximated with smoother surfaces, yielding more robust algorithm and more blurred motion field, typically poly_n =5 or 7.
- **`poly_sigma`**	standard deviation of the Gaussian that is used to smooth derivatives used as a basis for the polynomial expansion; for poly_n=5, you can set poly_sigma=1.1, for poly_n=7, a good value would be poly_sigma=1.5.
- **`flags`**	operation flags that can be a combination of the following:

  1. **OPTFLOW_USE_INITIAL_FLOW** uses the input flow as an initial flow approximation.
  2. **OPTFLOW_FARNEBACK_GAUSSIAN** uses the Gaussian winsize×winsize filter instead of a box filter of the same size for optical flow estimation; usually, this option gives z more accurate flow than with a box filter, at the cost of lower speed; normally, winsize for a Gaussian window should be set to a larger value to achieve the same level of robustness.


In [13]:
# Reading a sample video
cap = cv2.VideoCapture("media/M4/vtest.avi")
ret, frame = cap.read()

prev_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

# Our mask in which we will color code.
mask = np.zeros_like(frame)

# The saturation in final result must be maximum.
mask[...,1] = 255

while(1):
    
    ret, frame = cap.read()
    if not ret:
        break
        
    current_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    # Using Farneback's Optical Flow, this computes the magnitude and the angles of 2d Vectors
    flow = cv2.calcOpticalFlowFarneback(prev_frame, current_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # The First channel corresponds to the magnitude and the second to angle 
    magnitude, angle = cv2.cartToPolar(flow[...,0], flow[...,1])
    
    # Setting the hue according to the optical flow direction.
    mask[...,0] = angle*180/np.pi/2
    # Setting the Value channel to the optical flow magnitude normalized between 0-255
    mask[...,2] = cv2.normalize(magnitude,None,0,255,cv2.NORM_MINMAX)
    
    # Converting hsv to bgr so we can display the image with imshow.
    bgr = cv2.cvtColor(mask,cv2.COLOR_HSV2BGR)
    
    # Stack the original frame with the result
    final = cv2.addWeighted(frame,0.5, bgr, 0.5, 0)
    
    cv2.imshow('frame', final)
    k = cv2.waitKey(1) 
    if k == 27:
        break
    
    # At the end current frame becomes the previous frame for the next iteration.
    prev_frame = current_frame

                
cv2.destroyAllWindows()
cap.release()